In [9]:
%env OPENAI_API_KEY="something random so that the OpenAI constructor does not complain"

env: OPENAI_API_KEY="something random so that the OpenAI constructor does not complain"


In [17]:
with open("/home/work/mgjeon/NegotiationArena/experiment/buysell/gpt-3.5-turbo-1106_gpt-3.5-turbo-1106_Altruistic_Altruistic_run_1/1724598863826/game_state.json") as f:
    f

In [10]:
#@title Just Run This Code To Import Some Things


import sys
from dotenv import load_dotenv

sys.path.append('NegotiationArena/')

import os
import numpy as np
import pandas as pd
from datetime import datetime
import traceback

from negotiationarena.logging import GameDecoder
from negotiationarena.game_objects.game import Game
from games import *
from negotiationarena.constants import *
from games.buy_sell_game.game import BuySellGame

import json

ALL_CONSTANTS = [
    RESOURCES_TAG,
    GOALS_TAG,
    REASONING_TAG,
    PLAYER_ANSWER_TAG,
    PROPOSED_TRADE_TAG,
    MESSAGE_TAG,
    VALUATION_TAG,
    OTHER_PLAYER_PROPOSED_TRADE,
    OTHER_PLAYER_ANSWER,
    OTHER_PLAYER_MESSAGE,
]

def get_from_summary(key, game_state, default=None):
    if key in game_state.game_state[-1]["summary"]:
        return game_state.game_state[-1]["summary"][key]
    else:
        return default


def compute_game_summary(game_states):
    # print(game_states[0].game_state[-1]["summary"].keys())
    game_name = np.array([g.__class__.__name__ for g in game_states])[:, None]
    log_path = np.array([g.log_path for g in game_states])[:, None]
    models = np.array([[p.model for p in g.players] for g in game_states])
    beheaviour = np.array(
        [
            g.game_state[0]["settings"]["player_social_behaviour"]
            for g in game_states
        ]
    )
    valuations = np.array(
        [
            get_from_summary("player_valuation", g, default=[None, None])
            for g in game_states
        ]
    )
    initial_resources = np.array(
        [get_from_summary("player_initial_resources", g) for g in game_states]
    )
    final_resources = (
        np.array(
            [get_from_summary("final_resources", g) for g in game_states]
        ),
    )
    try:
        resources_delta = (final_resources - initial_resources)[0]
        resources_delta = np.array(
            [
                v.value(r) if v else r.value()
                for r, v in zip(
                    resources_delta.reshape(
                        -1,
                    ),
                    valuations.reshape(-1),
                )
            ]
        )
        resources_delta = resources_delta.reshape(-1, 2)
    except:
        resources_delta = np.array([0, 0] * len(game_states)).reshape(-1, 2)

    df = np.concatenate(
        (
            game_name,
            log_path,
            models,
            beheaviour,
            # outcomes.reshape(-1, 1),
            resources_delta,
        ),
        axis=1,
    )
    df = pd.DataFrame(
        df,
        columns=[
            "game_name",
            "log_path",
            "model_1",
            "model_2",
            "behaviour_1",
            "behaviour_2",
            "resource_delta_1",
            "resource_delta_2",
        ],
    )
    return df


def load_states_from_dir(log_dir: str):
    state_paths = sorted(
        [
            os.path.join(log_dir, f, "game_state.json")
            for f in os.listdir(log_dir)
        ]
    )
    game_states = []
    for path in state_paths:
        try:
            with open(path) as f:
                json_game = json.load(f, cls=GameDecoder)
                json_game["log_path"] = os.path.dirname(path)
                game = Game.from_dict(json_game)
                # we only want games which have ended
                assert (
                    game.game_state[-1]["current_iteration"] == "END"
                ), "WARNING : Game  {} has not ended\n".format(path)
                game_states.append(game)

        except Exception as e:
            exception_type = type(e).__name__
            exception_message = str(e)
            stack_trace = traceback.format_exc()

            # Print or use the information as needed
            print(f"\nException Type: {exception_type}")
            print(f"Exception Message: {exception_message}")
            print(f"Stack Trace:\n{stack_trace}")

    print("THERE ARE {} log files".format(len(state_paths)))
    print("{} Loaded Successfully".format(len(game_states)))
    return game_states


def from_timestamp_str(ts: str):
    return datetime.fromtimestamp(int(ts) // 1000)


def get_log_path_from_summary(selected_game, games_summary_df):
    game_to_load = (
        games_summary_df[games_summary_df["list_name"] == selected_game]
        .iloc[0]
        .log_path
    )
    return os.path.join(game_to_load, "game_state.json")


In [24]:
import os
import shutil

def move_files_and_remove_folder(src_dir, target_dir):
    for root, dirs, files in os.walk(src_dir):
        for file in files:
            # 현재 파일의 전체 경로
            file_path = os.path.join(root, file)
            
            # 중간의 불필요한 폴더를 제거하고 새로운 경로를 생성
            relative_path = os.path.relpath(file_path, src_dir)
            new_path = os.path.join(target_dir, os.path.basename(root), file)
            
            # 타겟 디렉토리가 없다면 생성
            os.makedirs(os.path.dirname(new_path), exist_ok=True)
            
            # 파일을 새로운 위치로 복사
            shutil.copy2(file_path, new_path)
            print(f"Copied {file_path} to {new_path}")

    print("All files copied successfully.")

# 사용 예시



src_dir = '/home/work/mgjeon/NegotiationArena/experiment_perosna/buysell/gpt-3.5-turbo-1106_gpt-3.5-turbo-1106_Altruistic_Altruistic_run_1/' 
for src_dir in os.listdir("/home/work/mgjeon/NegotiationArena/experiment_perosna/buysell/"):
    src = f"/home/work/mgjeon/NegotiationArena/experiment_perosna/buysell/{src_dir}"
    target_dir = '/home/work/mgjeon/NegotiationArena/experiment_cleaned_v2/buysell/cleaned/'
    move_files_and_remove_folder(src, target_dir)

Copied /home/work/mgjeon/NegotiationArena/experiment_perosna/buysell/gpt-4-1106-preview_gpt-3.5-turbo-1106_Altruistic_Cooperative_run_4/1724620386027/interaction.log to /home/work/mgjeon/NegotiationArena/experiment_cleaned_v2/buysell/cleaned/1724620386027/interaction.log
Copied /home/work/mgjeon/NegotiationArena/experiment_perosna/buysell/gpt-4-1106-preview_gpt-3.5-turbo-1106_Altruistic_Cooperative_run_4/1724620386027/game_state.json to /home/work/mgjeon/NegotiationArena/experiment_cleaned_v2/buysell/cleaned/1724620386027/game_state.json
All files copied successfully.
Copied /home/work/mgjeon/NegotiationArena/experiment_perosna/buysell/gpt-4-1106-preview_gpt-4-1106-preview_Cooperative_Desperate_run_2/1724626552101/interaction.log to /home/work/mgjeon/NegotiationArena/experiment_cleaned_v2/buysell/cleaned/1724626552101/interaction.log
Copied /home/work/mgjeon/NegotiationArena/experiment_perosna/buysell/gpt-4-1106-preview_gpt-4-1106-preview_Cooperative_Desperate_run_2/1724626552101/game_

In [21]:
os.listdir("/home/work/mgjeon/NegotiationArena/experiment_perosna/buysell/")

['gpt-4-1106-preview_gpt-3.5-turbo-1106_Altruistic_Cooperative_run_4',
 'gpt-4-1106-preview_gpt-4-1106-preview_Cooperative_Desperate_run_2',
 'gpt-4o-mini_gpt-4o_Competitive_Competitive_run_5',
 'gpt-4-1106-preview_gpt-3.5-turbo-1106_Cunning_Desperate_run_3',
 'gpt-3.5-turbo-1106_gpt-4o-mini_Competitive_Cunning_run_2',
 'gpt-4o_gpt-4o_Altruistic_Desperate_run_3',
 'gpt-3.5-turbo-1106_gpt-4-1106-preview_Competitive_Competitive_run_4',
 'gpt-3.5-turbo-1106_gpt-4o-mini_Cunning_Altruistic_run_2',
 'gpt-4o_gpt-4-1106-preview_Control_Desperate_run_3',
 'gpt-4o_gpt-4o-mini_Cunning_Selfish_run_5',
 'gpt-3.5-turbo-1106_gpt-4o-mini_Selfish_Desperate_run_5',
 'gpt-3.5-turbo-1106_gpt-4-1106-preview_Competitive_Cooperative_run_3',
 'gpt-4o-mini_gpt-4o_Competitive_Cunning_run_1',
 'gpt-4-1106-preview_gpt-3.5-turbo-1106_Cunning_Cooperative_run_2',
 'gpt-4o-mini_gpt-4-1106-preview_Cooperative_Control_run_1',
 'gpt-4o-mini_gpt-4o_Selfish_Desperate_run_1',
 'gpt-4o_gpt-3.5-turbo-1106_Competitive_Competi

In [25]:
# we load the states

states = load_states_from_dir("/home/work/mgjeon/NegotiationArena/experiment_cleaned_v2/buysell/cleaned/")


Exception Type: AssertionError
Exception Message: WARNING : Game  /home/work/mgjeon/NegotiationArena/experiment_cleaned_v2/buysell/cleaned/1724568300384/game_state.json has not ended

Stack Trace:
Traceback (most recent call last):
  File "/tmp/ipykernel_856526/3864640999.py", line 128, in load_states_from_dir
    game.game_state[-1]["current_iteration"] == "END"
AssertionError: WARNING : Game  /home/work/mgjeon/NegotiationArena/experiment_cleaned_v2/buysell/cleaned/1724568300384/game_state.json has not ended



Exception Type: AssertionError
Exception Message: WARNING : Game  /home/work/mgjeon/NegotiationArena/experiment_cleaned_v2/buysell/cleaned/1724569109245/game_state.json has not ended

Stack Trace:
Traceback (most recent call last):
  File "/tmp/ipykernel_856526/3864640999.py", line 128, in load_states_from_dir
    game.game_state[-1]["current_iteration"] == "END"
AssertionError: WARNING : Game  /home/work/mgjeon/NegotiationArena/experiment_cleaned_v2/buysell/cleaned/1724569109

In [26]:
# we build a dataframe

# resource_delta_2 contains the payoff for player 2, the buyer. In this context payoff
# is the willingness to pay for the buyer minus the sale price of the object.
# this means that if resource_delta_2 is positive, than the buyer has paid less than
# its what he was willing to spend.

# similar conclusions can be drawn for resource_delta_1, it's what the seller as gained above 40.
# in the games you see, seller does not gain much, usually.

df = compute_game_summary(states)
df["resource_delta_2"] = df["resource_delta_2"].astype(int)

In [30]:
df

,game_name,log_path,model_1,model_2,behaviour_1,behaviour_2,resource_delta_1,resource_delta_2
0,BuySellGame,/home/work/mgjeon/NegotiationArena/experiment_...,gpt-4o,gpt-4o,"You are a negotiator who prioritizes teamwork,...",You are a diplomatic negotiator who excels at ...,2,18
1,BuySellGame,/home/work/mgjeon/NegotiationArena/experiment_...,gpt-4o,gpt-4o,You are a community-minded negotiator who beli...,"You are a negotiator who prioritizes teamwork,...",10,10
2,BuySellGame,/home/work/mgjeon/NegotiationArena/experiment_...,gpt-4o,gpt-4o,You are a diplomatic negotiator who excels at ...,"You are a negotiator who prioritizes teamwork,...",15,5
3,BuySellGame,/home/work/mgjeon/NegotiationArena/experiment_...,gpt-4o,gpt-4o,"You are a negotiator who prioritizes teamwork,...","You are a negotiator who prioritizes teamwork,...",15,5
4,BuySellGame,/home/work/mgjeon/NegotiationArena/experiment_...,gpt-4o,gpt-4o,You are a community-minded negotiator who beli...,You are a community-minded negotiator who beli...,10,10
...,...,...,...,...,...,...,...,...
3750,BuySellGame,/home/work/mgjeon/NegotiationArena/experiment_...,gpt-4-1106-preview,gpt-4-1106-preview,,You are a participant in a Negotiation. Naviga...,7,13
3751,BuySellGame,/home/work/mgjeon/NegotiationArena/experiment_...,gpt-4-1106-preview,gpt-4-1106-preview,You are a helpful assistant in a negotiation s...,You are a helpful assistant in a negotiation s...,7,13
3752,BuySellGame,/home/work/mgjeon/NegotiationArena/experiment_...,gpt-4-1106-preview,gpt-4-1106-preview,,,6,14
3753,BuySellGame,/home/work/mgjeon/NegotiationArena/experiment_...,gpt-4-1106-preview,gpt-4-1106-preview,You are a participant in a Negotiation. Naviga...,You are a helpful assistant in a negotiation s...,7,13


In [29]:
df.to_excel('/home/work/mgjeon/NegotiationArena/persona_experiment.xlsx', index=False)

In [31]:
from personas import persona_dict_english_v2

In [37]:
persona_dict_english_v2

{'Cooperative': {'C0': 'You are a negotiator who prioritizes teamwork, always looking for ways to collaborate and achieve mutually beneficial outcomes. Aim to share resources and ideas to spend less ZUP.',
  'C1': 'You are a diplomatic negotiator who excels at bringing people together. Focus on fostering cooperation among all parties to reduce the ZUP expenditure.',
  'C2': 'You are a community-minded negotiator who believes in collective success. Encourage others to work together towards a common goal, aiming to minimize the ZUP spent.'},
 'Competitive': {'P0': 'You are a cutthroat negotiator who thrives on competition. Push aggressively to outdo your opponent, ensuring you spend the least ZUP possible.',
  'P1': 'You are a fierce negotiator driven by a desire to win. Relentlessly pursue victory in negotiations, aiming to lower your ZUP expenditure at all costs.',
  'P2': 'You are a highly ambitious negotiator who views every negotiation as a chance to assert dominance. Strategize to 

In [40]:
def find_persona(sentence) : 
    for persona in list(persona_dict_english_v2.keys()):
        for item in list(persona_dict_english_v2[persona].keys()):
            if sentence == persona_dict_english_v2[persona][item]:
                return {"persona" : persona, "item" : item}

In [47]:
lst = []
for i, row in df.iterrows():
    temp_dict = dict(row)
    temp = find_persona(temp_dict['behaviour_1'])
    temp_dict['behaviour_1_persona'] = temp['persona']
    temp_dict['behaviour_1_persona_item'] = temp['item']

    temp = find_persona(temp_dict['behaviour_2'])
    temp_dict['behaviour_2_persona'] = temp['persona']
    temp_dict['behaviour_2_persona_item'] = temp['item']

    lst.append(temp_dict)

In [50]:
pd.DataFrame(lst).to_excel('/home/work/mgjeon/NegotiationArena/persona_experiment_added_persona_item.xlsx', index=False)